# Stance Detection Using Dummy Models

`Dummy Models`  
`AraStance Dataset`  
`Stance Detection` `Arabic Language`

---

In this notebook, we test the performance of dummy models as baselines for the task of article stance detection in the AraStance dataset. The dataset was introduced in the paper:
```
AraStance: A Multi-Country and Multi-Domain Dataset of Arabic Stance Detection for Fact Checking.
```

## Packages

In [1]:
import numpy as np
from sklearn.metrics import f1_score
from utils import AraStanceData, stance_to_int

## Raw data

- Download the raw data:

In [ ]:
!wget https://github.com/Tariq60/arastance/archive/refs/heads/main.zip
!unzip /content/main.zip

- Let's start by reading the raw data:

In [2]:
raw_train = AraStanceData("/content/arastance-main/data/train.jsonl")
raw_val = AraStanceData("/content/arastance-main/data/dev.jsonl")
raw_test = AraStanceData("/content/arastance-main/data/test.jsonl")

print(f'# training instances: {len(raw_train.stances)}')
print(f'# validation instances: {len(raw_val.stances)}')
print(f'# testing instances: {len(raw_test.stances)}')

# training instances: 2848
# validation instances: 569
# testing instances: 646


- Let's see an instance from the data:

In [3]:
instance_no = 110
print(f"Claim text: {raw_train.claims[raw_train.article_claim[instance_no]]}")
print(f"Article text: {raw_train.articles[instance_no]}")
print(f"Stance: {raw_train.stances[instance_no]}")

Claim text: صدور بيان بقائمة أسماء ممنوعة ووجوب تغيير الاسم خلال ثلاثين يوم.
Article text: ماحقيقة الاسماء الممنوعة في السعودية ، سؤال تداوله الجميع عبر مواقع التواصل الاجتماعي بعد أن تناقل الكثيرون قائمة بأسماء منعتها الأحوال المدنية في السعودية شملت الكثير من الأسماء التي قد لايخلو منزل منها ، وتساءل البعض عن أسباب المنع وكيف يمكن تغيير الاسماء التي تم تسميتها مسبقا ؟ الأحوال المدنية توضح حقيقة الاسماء الممنوعة في السعودية أوضحت الأحوال المدنية السعودية أن ما يتم تناقله عبر المواقع الاجتماعية عن أسماء ممنوعة في السعودية هي معلومات خالية من الصحة ، وقالت عبر حسابها الرسمي على تويتر ( ما يتم تداوله حول صدور بيان بقائمة أسماء ممنوعة ووجوب تغيير الاسم خلال ثلاثين يوماً غير صحيح، ونأمل أخذ المعلومات من المصادر الرسمية. كما وأوضح المتحدث الرسمي للأحوال المدنية محمد الجاسر أنه لا يتم تسجيل أسماء المواليد غير الجائزة شرعا والأسماء غير الملائمة اجتماعيا ونحوها، وأضاف أن اللائحة التنفيذية لنظام الأحوال المدنية وضحت أنه لا يجوز التسمية بالأسماء التي نصت الفتاوى الشرعية على عدم جوازها، إضافة الأ

## Random classifier

- Let's start with the performance of a random classifier that gives random predictions:

In [4]:
def random_classifier(labels, no_trials = 5):

  accuracy, mf1score, f1score = 0, 0, 0
  no_classes = len(np.unique(labels))

  for _ in range(no_trials):
    predictions = np.array([np.random.randint(0, no_classes) for _ in range(len(labels))])

    accuracy = np.sum(predictions == np.array(labels)) / len(labels) * no_trials

    f1score += f1_score(labels, predictions, average=None) / no_trials
    mf1score += f1_score(labels, predictions, average='macro') / no_trials

  return accuracy, f1score, mf1score

In [8]:
test_labels = [stance_to_int[stance] for stance in raw_test.stances]
accuracy, f1score, mf1score= random_classifier(test_labels, 1)
print("Accuracy                   :", accuracy)
print("F1 score                   :", f1score)
print("Macro F1 score             :", mf1score)

Accuracy                   : 0.2693498452012384
F1 score                   : [0.25705329 0.1888412  0.17167382 0.35897436]
Macro F1 score             : 0.24413566799240916


## Majority classifier

- Now, let's check the performance of a majority classifier that predicts the majority class all the time:

In [6]:
def majority_classifier(labels, majority_class):

  predictions = [stance_to_int[majority_class]] * len(labels)
  accuracy = np.sum(np.array(predictions) == np.array(labels)) / len(labels)

  f1score = f1_score(labels, predictions, average=None)
  mf1score = f1_score(labels, predictions, average='macro')

  return accuracy, f1score, mf1score

In [7]:
test_labels = [stance_to_int[stance] for stance in raw_test.stances]
accuracy, f1score, mf1score = majority_classifier(test_labels, 'unrelated')
print("Accuracy                   :", accuracy)
print("F1 score                   :", f1score)
print("Macro F1 score             :", mf1score)

Accuracy                   : 0.5541795665634675
F1 score                   : [0.         0.         0.         0.71314741]
Macro F1 score             : 0.17828685258964144
